In [53]:
!adb exec-out run-as si.fri.matevzfa.approxhpvmdemo cat databases/classification-log > classification-log.sqlite3

In [54]:
import sqlite3
import pandas as pd
import ipywidgets as widgets

con = sqlite3.connect("classification-log.sqlite3")

df_full = pd.read_sql_query("SELECT * FROM classification", con)
df_full["timestamp"] = pd.to_datetime(df_full["timestamp"], infer_datetime_format=True)
df_full["run_start"] = pd.to_datetime(df_full["run_start"], infer_datetime_format=True)

df_full["run_start"].unique()

w = widgets.Select(
    options=df_full["run_start"].unique(),
    description="Select run"
)
w

Select(description='Select run', options=(Timestamp('2021-08-19 17:21:29.422000+0200', tz='pytz.FixedOffset(12…

In [173]:
import matplotlib.pyplot as plt
import numpy as np

labels = [
    "Walking",
    "W. Upstairs",
    "W. Downstairs",
    "Sitting",
    "Standing",
    "Lying",
]

df = df_full[df_full["run_start"] == w.value].copy()
df["timestamp"] = (df["timestamp"] - df["timestamp"].min()) / pd.to_timedelta(1, unit='s')

perrow_cca = 100
nrows = int(np.ceil(len(df) / perrow_cca))

perrow_seconds = df.timestamp.max() / nrows

fig, axs = plt.subplots(nrows=nrows, figsize=(7, nrows*2.2))
plt.subplots_adjust(hspace=0.3)

plotted = 0

for i, ax1 in enumerate(axs):

    ts_min = i*perrow_seconds
    ts_max = (i+1)*perrow_seconds

    df_sub = df.loc[(ts_min <= df.timestamp) & (df.timestamp <= ts_max), :]

    plotted += len(df_sub)

    ax2 = ax1.twinx()

    ln_conf = ax2.plot(df_sub.timestamp, df_sub.used_config, 'k:', label="Used approx. config.")
    ax2.set_yticks(list(range(10)))
    ax2.set_yticklabels(list(range(10)))

    wrong = df_sub[df_sub.argmax != df_sub.argmax_baseline]

    for ts in wrong.timestamp:
        ax1.axvline(ts, c='gray', ls='--', lw=1)

    ax1.grid(ls=':', axis='y')
    ln_r = ax1.plot(wrong.timestamp, wrong.argmax, 'r.', label="Different result to baseline")
    ln_g = ax1.plot(df_sub.timestamp, df_sub.argmax_baseline, 'g.', label="Baseline")
    ax1.set_yticks(list(range(len(labels))))
    ax1.set_yticklabels(labels)

    ax1.set_xlim(ts_min-1, ts_max+1)

    ax1.set_ylabel("Activity")
    ax2.set_ylabel("Approx. config.")

lns = ln_conf + ln_r + ln_g
axs[-1].legend(lns, [l.get_label() for l in lns],
               loc="center", bbox_to_anchor=[0.5, -0.6], ncol=3)

axs[-1].set_xlabel("Time [s]")

fig.savefig(f"adaptation-{str(w.value).replace(' ', '-')}.pdf", bbox_inches='tight')
plt.show()

print(f"missing: {plotted - len(df)}")
